In [ ]:
!pip install langchain_community pypdf chromadb google-generativeai
!!pip install -U langchain-google-genai faiss-cpu

['Collecting langchain-google-genai',
 '  Downloading langchain_google_genai-2.0.9-py3-none-any.whl.metadata (3.6 kB)',
 'Collecting faiss-cpu',
 '  Downloading faiss_cpu-1.9.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)',
 'Collecting filetype<2.0.0,>=1.2.0 (from langchain-google-genai)',
 '  Downloading filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)',
 'Requirement already satisfied: google-generativeai<0.9.0,>=0.8.0 in /usr/local/lib/python3.11/dist-packages (from langchain-google-genai) (0.8.3)',
 'Requirement already satisfied: langchain-core<0.4.0,>=0.3.27 in /usr/local/lib/python3.11/dist-packages (from langchain-google-genai) (0.3.31)',
 'Requirement already satisfied: pydantic<3,>=2 in /usr/local/lib/python3.11/dist-packages (from langchain-google-genai) (2.10.5)',
 'Requirement already satisfied: numpy<3.0,>=1.25.0 in /usr/local/lib/python3.11/dist-packages (from faiss-cpu) (1.26.4)',
 'Requirement already satisfied: packaging in 

In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import Chroma
import pandas as pd
df = pd.read_csv('aboutEnsah.csv', header=None)
df.to_csv('aboutEnsah.csv', index=False)
loader = CSVLoader(
    "aboutEnsah.csv" ,
)
docs = loader.load()
docs[0]

Document(metadata={'source': 'aboutEnsah.csv', 'row': 0}, page_content="0: cp = Cycle Préparatoire  = ap\n1: cordinnateur  du  cycle preparatoire: Pr. Ahmed BOUJRAF\n2: email de cordinnateur  du  cycle preparatoire : a.boujraf@uae.ac.ma\n3: Cycle Préparatoire : Sciences & Techniques pour l’Ingénieur Objectifs de la formation : Préparer et donner à l’élève ingénieur les bases scientifiques et techniques, ainsi que les connaissances en langues et communications indispensables à la poursuite des études dans une filière du cycle ingénieur. Le cursus suivi au cycle préparatoire de l’ENSA d’Al-Hoceima ainsi que celui suivi dans less filières accréditées par l’ENSAH permettent de: Répondre à la demande régionale et nationale en matière de ressources humaines qualifiées dans les domaines du Génie Civil, de l’Environnement, de l'Enérgitique et de l’Informatique. Renforcer la compétitivité et le développement socio-économique régional et national. Compétences à acquérir : Le cycle préparatoire i

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
text_splitter.split_documents(docs)[:3]

[Document(metadata={'source': 'aboutEnsah.csv', 'row': 0}, page_content='0: cp = Cycle Préparatoire  = ap\n1: cordinnateur  du  cycle preparatoire: Pr. Ahmed BOUJRAF\n2: email de cordinnateur  du  cycle preparatoire : a.boujraf@uae.ac.ma'),
 Document(metadata={'source': 'aboutEnsah.csv', 'row': 0}, page_content="3: Cycle Préparatoire : Sciences & Techniques pour l’Ingénieur Objectifs de la formation : Préparer et donner à l’élève ingénieur les bases scientifiques et techniques, ainsi que les connaissances en langues et communications indispensables à la poursuite des études dans une filière du cycle ingénieur. Le cursus suivi au cycle préparatoire de l’ENSA d’Al-Hoceima ainsi que celui suivi dans less filières accréditées par l’ENSAH permettent de: Répondre à la demande régionale et nationale en matière de ressources humaines qualifiées dans les domaines du Génie Civil, de l’Environnement, de l'Enérgitique et de l’Informatique. Renforcer la compétitivité et le développement socio-écono

In [ ]:
documents= text_splitter.split_documents(docs)

In [ ]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents,GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

llm = ChatGoogleGenerativeAI(model="gemini-pro" , generation_config=genai.GenerationConfig(
        temperature=0.1,
    ))
llm

ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x797274f86050>, default_metadata=())

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are an AI Agent specialized in answering questions about ENSAH.
Your role is to provide accurate and helpful answers about the programs, professors, and training offered at these institutions.

Use the provided context to create your response:
- Refer only to the most relevant information from the context list to answer questions about ENSAH.
- Explain your response using simple and clear words, avoiding abrupt or short answers.
- Always respond in the same language used in the question, ensuring politeness and professionalism.
- If the question cannot be answered using the context or is unrelated, respond with 'thella.'

<context>
{context}
</context>

Question: {input}
""")



In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x797274aa8c10>, search_kwargs={})

In [ ]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response=retrieval_chain.invoke({"input":" je  suis  entrain de choisir  entre gc et gi , donne  moi une idee sur les deux , donne moi les objectifs de chaque  filiere  "})


In [ ]:
response['answer']

"**Génie Civil (GC)**\n\n* **Objectifs de la formation:** Non spécifiés dans le contexte fourni.\n\n**Génie Informatique (GI)**\n\n* **Objectifs de la formation:**\n    * Former des ingénieurs polyvalents avec des compétences scientifiques et techniques en informatique.\n    * Maîtriser les concepts et technologies dans les domaines suivants:\n        * Modélisation et programmation\n        * Bases de données\n        * Systèmes et réseaux\n        * Conception et déploiement d'applications d'entreprise\n        * Sécurité informatique\n        * Prise de décision\n        * Analyse de données\n        * Visualisation des données\n    * Préparer les étudiants au monde de l'entreprise grâce à des modules de management, de marketing et de communication."

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory


In [ ]:
reformulation_prompt = ChatPromptTemplate.from_template("""
Given a chat history and the latest user question, which might reference earlier messages in the conversation:
- Reformulate the user's question into a standalone question that can be understood without the chat history.
- If no reformulation is needed, use the question as is. Do not provide an answer.

### Chat History:
{chat_history}

### User Question:
{input}

### Reformulated Question:
""")

In [ ]:
store = {}


In [ ]:
store

{'my_session': InMemoryChatMessageHistory(messages=[HumanMessage(content='Quelle est le nom du coordinateur du filière TDIA?', additional_kwargs={}, response_metadata={}), AIMessage(content='Le coordinateur du filière TDIA est ABDELKHALAK BAHRI.', additional_kwargs={}, response_metadata={}), HumanMessage(content='qulle est sont  email', additional_kwargs={}, response_metadata={}), AIMessage(content='thella', additional_kwargs={}, response_metadata={}), HumanMessage(content='quelle est sont email ??', additional_kwargs={}, response_metadata={}), AIMessage(content='thella', additional_kwargs={}, response_metadata={}), HumanMessage(content='Quelle est le nom du coordinateur du filière TDIA?', additional_kwargs={}, response_metadata={}), AIMessage(content='Le coordinateur de la filière TDIA est ABDELKHALAK BAHRI.', additional_kwargs={}, response_metadata={}), HumanMessage(content='quelle est sont email ??', additional_kwargs={}, response_metadata={}), AIMessage(content='thella', additional

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory

# Dictionary to store chat histories
chat_histories = {}

def get_session_history(session_id):
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]


In [ ]:
from langchain_core.runnables import RunnableWithMessageHistory

conversational_rag_chain = RunnableWithMessageHistory(
    runnable=retrieval_chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)


In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "Quelle est le nom du coordinateur du filière TDIA?"},
    config={"configurable": {"session_id": "my_session"}}
)["answer"]

In [ ]:
print(response)


Le coordinateur de la filière TDIA est ABDELKHALAK BAHRI.


In [ ]:
conversational_rag_chain.invoke(
    {"input": "quelle est sont email ??"},
    config={"configurable": {"session_id": "my_session"}}
)["answer"]

'thella'